In [92]:
import json
import requests
import time
import csv

In [93]:
group_id = '-92616631'
access_token = 'XXXXXXXXX'
deadline = 1577826000

Get IDs of needed publications not older than „deadline“

In [94]:
ids = []
while len(ids)%100==0:
    url = 'https://api.of.service.com/method/mtd.get?owner_id=' + group_id + '&count=100&offset=' + str(len(ids)) + '&access_token=' + access_token + '&v=1.111'
    response = requests.get(url)
    data = json.loads(response.text)
    for post in data['response']['items']:
        if post['date'] > deadline:
            ids.append(post['id'])

print(len(ids))

266


Get texts from publications not older than „deadline“

In [ ]:
comments = []
rsp_count = 0

for postid in ids:
    len_diff = 0
    len_before = len(comments)
    print('id = ', postid)
    while len_diff%100 == 0:
        url = 'https://api.of.service.com/method/mtd.getTexts?owner_id=' + group_id + '&post_id=' + str(postid) + '&preview_length=0&count=100&offset=' + str(len_diff) + '&access_token=' + access_token + '&v=1.111'
        rsp_count +=1
        if rsp_count%5 == 0:
            print('! TIMEOUT !')
            time.sleep(2)
        response_c = requests.get(url)
        ## print(response_c.text)
        post_comments = json.loads(response_c.text)
        if len(post_comments['response']['items']) == 0:
            break
        for comment in post_comments['response']['items']:
            if comment['date'] > deadline:
                textaslist = [comment['text'].replace('\n', ' ')]
                comments.append(textaslist)

        len_diff = len(comments) - len_before
        print('diff = ', len_diff)

print(len(comments))

In [96]:
file = open('/group_texts.csv', 'w')
wr = csv.writer(file)
for comm in comments:
    wr.writerow(comm)

file.close()